# Hyperparameter tuning via FLAML #

### Sukanta Basu (Delft University of Technology) ###

**Last updated: November 3, 2022**

Notes: 

1. 

**Load the necessary packages**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

import pickle
from pickle import dump, load
import time

from flaml import AutoML

#For reproducibility of the results, the following seeds should be selected 
from numpy.random import seed
seed(20)
randSeed = np.random.randint(1000)

/Users/sukantabasu/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


**User input**

In [2]:
#Number of ensembles
nEns    = 25

#Model
mod     = ['lgbm', 'xgboost', 'xgb_limitdepth', 'catboost', 'rf', 'extra_tree']
modOpt  = 3 #[0:'lgbm', 1:'xgboost', 2:'xgb_limitdepth', 3:'catboost', 4:'rf', 5:'extra_tree'],

#Time budget
maxTime = 60 #in seconds

#Cn2 is measured at 6 m, 15 m, and 25 m
zLev    = 6 

#1:ERA5, 2:MERRA2, 3:COMBO
inpOpt  = 3 

#Sampling opt
sOpt = 1 #0: hourly, 1: 5 min

#Train-validation ratio
split_ratio = 0.2

**Input & output directories**

In [3]:
ROOT_DIR = "/Users/sukantabasu/Dropbox/Priority/Works_Ongoing/AIML/2022_HawaiiCn2Reanalysis/"

INPUT_DIR  = ROOT_DIR + "ExtractedDATA/v02/Tuning/"
OUTPUT_DIR = ROOT_DIR + "ExtractedDATA/v02/Tuning/" 

In [4]:
if modOpt == 0:
    TUNING_DIR  = OUTPUT_DIR + 'LGBM/'
elif modOpt == 1:
    TUNING_DIR  = OUTPUT_DIR + 'XGB/'
elif modOpt == 2:
    TUNING_DIR  = OUTPUT_DIR + 'XGBL/'
elif modOpt == 3:
    TUNING_DIR  = OUTPUT_DIR + 'CAT/'
elif modOpt == 4:
    TUNING_DIR  = OUTPUT_DIR + 'RF/'
elif modOpt == 5:
    TUNING_DIR  = OUTPUT_DIR + 'EXT/'    

#### FLAML Configuration

In [5]:
automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": maxTime,  # in seconds
    "metric": 'mse',
    "task": 'regression',
    "estimator_list": [mod[modOpt]],
    "eval_method":'holdout',
    "early_stop": True,
    "sample": True, #A boolean of whether to sample the training data during search
    "ensemble": False, #Whether to perform ensemble after search
    "model_history": True, #A boolean of whether to keep the best model per estimator
    "retrain_full": True, #whether to retrain the selected model on the full training data
    "log_file_name": 'FLAML_' + str(modOpt) + '.log'
}

**Training, validation data**

In [6]:
if inpOpt == 1:

    if sOpt == 0:
        df_F_TrnVal = pd.read_csv(OUTPUT_DIR+'TrnVal_ERA5_60min.csv')
    else:
        df_F_TrnVal = pd.read_csv(OUTPUT_DIR+'TrnVal_ERA5_5min.csv')    
    XTrnVal = df_F_TrnVal[['sinDY','cosDY','sinHR','cosHR','ERA5_WSPD_10m','ERA5_WSPD_100m','ERA5_GUST_10m','ERA5_alpha','ERA5_beta','ERA5_T_2m','ERA5_TSK','ERA5_TSL','ERA5_Td_2m','ERA5_dT1','ERA5_dT2','ERA5_dT3','ERA5_UST','ERA5_SHFX','ERA5_LH','ERA5_PMSL','ERA5_PBLH','ERA5_TCC','ERA5_LCC','ERA5_EDR','ERA5_CAPE']].values

elif inpOpt == 2: 
    
    if sOpt == 0:
        df_F_TrnVal = pd.read_csv(OUTPUT_DIR+'TrnVal_MERRA2_60min.csv')
    else:
        df_F_TrnVal = pd.read_csv(OUTPUT_DIR+'TrnVal_MERRA2_5min.csv')
            
    XTrnVal = df_F_TrnVal[['sinDY','cosDY','sinHR','cosHR','MRA2_WSPD_2m','MRA2_WSPD_10m','MRA2_WSPD_50m','MRA2_GUST','MRA2_alpha1','MRA2_alpha2','MRA2_beta1','MRA2_beta2','MRA2_T_2m','MRA2_T_10m','MRA2_TSK','MRA2_dT1','MRA2_dT2','MRA2_Q_2m','MRA2_Q_10m','MRA2_dQ','MRA2_UST','MRA2_SHFX','MRA2_LH','MRA2_PMSL','MRA2_PBLH','MRA2_RHO','MRA2_RIB']].values    

elif inpOpt == 3:
    
    if sOpt == 0:
        df_F_TrnVal = pd.read_csv(OUTPUT_DIR+'TrnVal_COMBO_60min.csv')
    else:
        df_F_TrnVal = pd.read_csv(OUTPUT_DIR+'TrnVal_COMBO_5min.csv')
            
    XTrnVal = df_F_TrnVal[['sinDY','cosDY','sinHR','cosHR','ERA5_WSPD_10m','ERA5_WSPD_100m','ERA5_GUST_10m','ERA5_alpha','ERA5_beta','ERA5_T_2m','ERA5_TSK','ERA5_TSL','ERA5_Td_2m','ERA5_dT1','ERA5_dT2','ERA5_dT3','ERA5_UST','ERA5_SHFX','ERA5_LH','ERA5_PMSL','ERA5_PBLH','ERA5_TCC','ERA5_LCC','ERA5_EDR','ERA5_CAPE','MRA2_WSPD_2m','MRA2_WSPD_10m','MRA2_WSPD_50m','MRA2_GUST','MRA2_alpha1','MRA2_alpha2','MRA2_beta1','MRA2_beta2','MRA2_T_2m','MRA2_T_10m','MRA2_TSK','MRA2_dT1','MRA2_dT2','MRA2_Q_2m','MRA2_Q_10m','MRA2_dQ','MRA2_UST','MRA2_SHFX','MRA2_LH','MRA2_PMSL','MRA2_PBLH','MRA2_RHO','MRA2_RIB']].values    
    
if zLev == 6:
    yTrnVal = df_F_TrnVal[['LCn2_06m']].values
elif zLev == 15: 
    yTrnVal = df_F_TrnVal[['LCn2_15m']].values
elif zLev == 25:
    yTrnVal = df_F_TrnVal[['LCn2_25m']].values

#### FLAML-based tuning

In [7]:
nSamples = np.size(yTrnVal)

for n in range(nEns):

    nSamplesVal = int(np.rint(split_ratio*nSamples))

    #Pick a random start location for validation set
    iVal        = np.random.randint(0,nSamples-nSamplesVal+1)

    print((nSamples,nSamplesVal,iVal))

    XVal        = XTrnVal[iVal:iVal+nSamplesVal+1,:]
    yVal        = yTrnVal[iVal:iVal+nSamplesVal+1,:]

    XTrn1       = XTrnVal[0:iVal,:]
    yTrn1       = yTrnVal[0:iVal,:]

    XTrn2       = XTrnVal[iVal+nSamplesVal+1:,:]
    yTrn2       = yTrnVal[iVal+nSamplesVal+1:,:]

    XTrn        = np.vstack((XTrn1, XTrn2))
    yTrn        = np.vstack((yTrn1, yTrn2))

    automl.fit(X_train=XTrn, y_train = yTrn.squeeze(), X_val = XVal, y_val = yVal.squeeze(), **automl_settings)

#   print(automl.best_config_per_estimator)
    print('Best ML leaner:', automl.best_estimator)
    print('Best hyperparmeter config:', automl.best_config)
    print('Best accuracy on validation data: {0:.4g}'.format(automl.best_loss))
    print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

    fSTR = TUNING_DIR + 'FLAML' + '_zLev' + str(zLev) + '_inpOpt' + str(inpOpt) + '_sOpt' + str(sOpt) + '_modOpt' + str(modOpt) + '_maxT' + str(maxTime) + '_Ens' + str(n) + '.pkl'
    with open(fSTR, "wb") as f:
        dump(automl, f, pickle.HIGHEST_PROTOCOL)

[flaml.automl: 11-03 19:04:31] {2600} INFO - task = regression
[flaml.automl: 11-03 19:04:31] {2602} INFO - Data split method: uniform
[flaml.automl: 11-03 19:04:31] {2605} INFO - Evaluation method: holdout
[flaml.automl: 11-03 19:04:31] {2727} INFO - Minimizing error metric: mse
[flaml.automl: 11-03 19:04:31] {2869} INFO - List of ML learners in AutoML Run: ['catboost']
[flaml.automl: 11-03 19:04:31] {3164} INFO - iteration 0, current learner catboost


(9312, 1862, 2522)


[flaml.automl: 11-03 19:04:33] {3297} INFO - Estimated sufficient time budget=18426s. Estimated necessary time budget=18s.
[flaml.automl: 11-03 19:04:33] {3344} INFO -  at 1.8s,	estimator catboost's best error=0.1384,	best estimator catboost's best error=0.1384
[flaml.automl: 11-03 19:04:33] {3164} INFO - iteration 1, current learner catboost
[flaml.automl: 11-03 19:04:34] {3344} INFO -  at 2.8s,	estimator catboost's best error=0.1384,	best estimator catboost's best error=0.1384
[flaml.automl: 11-03 19:04:34] {3164} INFO - iteration 2, current learner catboost
[flaml.automl: 11-03 19:04:35] {3344} INFO -  at 3.4s,	estimator catboost's best error=0.1384,	best estimator catboost's best error=0.1384
[flaml.automl: 11-03 19:04:35] {3164} INFO - iteration 3, current learner catboost
[flaml.automl: 11-03 19:04:36] {3344} INFO -  at 4.8s,	estimator catboost's best error=0.1384,	best estimator catboost's best error=0.1384
[flaml.automl: 11-03 19:04:36] {3164} INFO - iteration 4, current learne

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.09999999999999996, 'n_estimators': 322}
Best accuracy on validation data: 0.1384
Training duration of best run: 1.843 s
(9312, 1862, 4367)


[flaml.automl: 11-03 19:04:51] {3297} INFO - Estimated sufficient time budget=9477s. Estimated necessary time budget=9s.
[flaml.automl: 11-03 19:04:51] {3344} INFO -  at 1.0s,	estimator catboost's best error=0.2016,	best estimator catboost's best error=0.2016
[flaml.automl: 11-03 19:04:51] {3164} INFO - iteration 1, current learner catboost
[flaml.automl: 11-03 19:04:52] {3344} INFO -  at 2.5s,	estimator catboost's best error=0.2016,	best estimator catboost's best error=0.2016
[flaml.automl: 11-03 19:04:52] {3164} INFO - iteration 2, current learner catboost
[flaml.automl: 11-03 19:04:53] {3344} INFO -  at 3.4s,	estimator catboost's best error=0.1947,	best estimator catboost's best error=0.1947
[flaml.automl: 11-03 19:04:53] {3164} INFO - iteration 3, current learner catboost
[flaml.automl: 11-03 19:04:54] {3344} INFO -  at 4.5s,	estimator catboost's best error=0.1947,	best estimator catboost's best error=0.1947
[flaml.automl: 11-03 19:04:54] {3164} INFO - iteration 4, current learner 

...

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 17, 'learning_rate': 0.04232015103948017, 'n_estimators': 1082}
Best accuracy on validation data: 0.1909
Training duration of best run: 3.419 s
(9312, 1862, 3262)


[flaml.automl: 11-03 23:28:07] {3297} INFO - Estimated sufficient time budget=11744s. Estimated necessary time budget=12s.
[flaml.automl: 11-03 23:28:07] {3344} INFO -  at 1.2s,	estimator catboost's best error=0.1679,	best estimator catboost's best error=0.1679
[flaml.automl: 11-03 23:28:07] {3164} INFO - iteration 1, current learner catboost
[flaml.automl: 11-03 23:28:08] {3344} INFO -  at 2.4s,	estimator catboost's best error=0.1422,	best estimator catboost's best error=0.1422
[flaml.automl: 11-03 23:28:08] {3164} INFO - iteration 2, current learner catboost
[flaml.automl: 11-03 23:28:09] {3344} INFO -  at 3.5s,	estimator catboost's best error=0.1422,	best estimator catboost's best error=0.1422
[flaml.automl: 11-03 23:28:09] {3164} INFO - iteration 3, current learner catboost
[flaml.automl: 11-03 23:28:11] {3344} INFO -  at 5.5s,	estimator catboost's best error=0.1422,	best estimator catboost's best error=0.1422
[flaml.automl: 11-03 23:28:11] {3164} INFO - iteration 4, current learne

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.06233639237958607, 'n_estimators': 391}
Best accuracy on validation data: 0.1422
Training duration of best run: 1.221 s
(9312, 1862, 5264)


[flaml.automl: 11-03 23:28:40] {3297} INFO - Estimated sufficient time budget=10301s. Estimated necessary time budget=10s.
[flaml.automl: 11-03 23:28:40] {3344} INFO -  at 1.0s,	estimator catboost's best error=0.2228,	best estimator catboost's best error=0.2228
[flaml.automl: 11-03 23:28:40] {3164} INFO - iteration 1, current learner catboost
[flaml.automl: 11-03 23:28:42] {3344} INFO -  at 3.0s,	estimator catboost's best error=0.2228,	best estimator catboost's best error=0.2228
[flaml.automl: 11-03 23:28:42] {3164} INFO - iteration 2, current learner catboost
[flaml.automl: 11-03 23:28:43] {3344} INFO -  at 3.8s,	estimator catboost's best error=0.2228,	best estimator catboost's best error=0.2228
[flaml.automl: 11-03 23:28:43] {3164} INFO - iteration 3, current learner catboost
[flaml.automl: 11-03 23:28:44] {3344} INFO -  at 5.5s,	estimator catboost's best error=0.2147,	best estimator catboost's best error=0.2147
[flaml.automl: 11-03 23:28:44] {3164} INFO - iteration 4, current learne

Best ML leaner: catboost
Best hyperparmeter config: {'early_stopping_rounds': 10, 'learning_rate': 0.04521546790973537, 'n_estimators': 735}
Best accuracy on validation data: 0.2125
Training duration of best run: 2.297 s
